In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

output_parser = StrOutputParser()
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()

# Langchain Memory Test

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [39]:
demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [40]:
chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 39, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-23e1ff5d-cf5a-41db-8959-740d707a27af-0')

In [41]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='You just asked me to translate the sentence "I love programming" from English to French.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 74, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f44df3c-69b7-43fa-9fa2-ebd5972522b4-0')

# Langchain vectorizing document test

In [15]:
loader = WebBaseLoader("https://www.enterprisesg.gov.sg/financial-support/enterprise-development-grant")

docs = loader.load()

In [16]:
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [18]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
#retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response = retrieval_chain.invoke({"input": "How do I check if I'm eligible for the Enterprice Development grant?"})
print(response["answer"])

To check if you are eligible for the Enterprise Development Grant, you need to ensure that your business entity is registered and operating in Singapore, has at least 30% local equity held by Singaporean(s) and/or Singapore PR(s), and is financially ready to start and complete the project. Additionally, you can refer to the eligibility criteria mentioned in the provided context for more details.


In [23]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [24]:
chat_history = [HumanMessage(content="Can Singaporean companies apply for the EDG?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Enterprise Development Grant (EDG)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nx \n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisation\n\n\nTalent Attraction & Development\n\n\nQuality & Standards\n\n\nSustainability\n\n\n\n\nInnovate with Us\n\nOverview\n\n\nInnovation Capabilities\n\n\nInnovation Talent\n\n\nMarket Access & Networks\n\n\n\n\nGo Global\n\nOverview\n\n\nMarket Guides\n\n\nMarket Entry Support\n\n\nInternational Agreements\n\n\n\n\nPartner with Singapore\n\nWhy Singapore\n\n\nInnovation & Startups\n\n\nTrade\n\n\nInfrastructure\n\n\n\n\nFake Navigation Item\n\n\n\n\nIndustries\n\n\nFinancial Support\n\n\nResources\n\n\nAbout Us\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisati

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)


retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [28]:
chat_history = [HumanMessage(content="Can Singaporean companies apply for the EDG?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can Singaporean companies apply for the EDG?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Enterprise Development Grant (EDG)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nx \n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisation\n\n\nTalent Attraction & Development\n\n\nQuality & Standards\n\n\nSustainability\n\n\n\n\nInnovate with Us\n\nOverview\n\n\nInnovation Capabilities\n\n\nInnovation Talent\n\n\nMarket Access & Networks\n\n\n\n\nGo Global\n\nOverview\n\n\nMarket Guides\n\n\nMarket Entry Support\n\n\nInternational Agreements\n\n\n\n\nPartner with Singapore\n\nWhy Singapore\n\n\nInnovation & Startups\n\n\nTrade\n\n\nInfrastructure\n\n\n\n\nFake Navigation Item\n\n\n\n\nIndustries\n\n\nFinancial Support\n\n\nResources\n\n\nAbout Us\n\n

# Langchain integrate website parsing with chat message history

In [53]:
loader = WebBaseLoader("https://www.enterprisesg.gov.sg/financial-support/enterprise-development-grant")

docs = loader.load()

In [54]:
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [55]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [56]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert on government policies in Singapore, answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [57]:
demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [58]:
chain_with_message_history.invoke(
    {"input": "Tell me what are the conditions to be eligible for the EDG grant."},
    {"configurable": {"session_id": "foo"}},
)

AIMessage(content="To be eligible for the Enterprise Development Grant (EDG) in Singapore, a company must be registered and operating in the country, have a minimum of 30% local shareholding, and be in a financially viable position. Additionally, the company should have a clear project scope and meet the criteria for the specific EDG pillar they are applying for: namely, Core Capabilities, Innovation and Productivity, or Market Access. It's important to note that the EDG is available to support projects that help companies upgrade, innovate, or venture overseas, and the specific eligibility criteria may vary for each pillar.", response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 46, 'total_tokens': 167}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b821e1e-fd2b-491f-bb53-11d34839dbec-0')

In [59]:
chain_with_message_history.invoke(
    {"input": "Repeat what I just asked you."},
    {"configurable": {"session_id": "foo"}},
)

AIMessage(content='The conditions to be eligible for the Enterprise Development Grant (EDG) in Singapore include the company being registered and operating in the country, having a minimum of 30% local shareholding, being in a financially viable position, having a clear project scope, and meeting the specific criteria for the EDG pillar they are applying for, such as Core Capabilities, Innovation and Productivity, or Market Access.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 182, 'total_tokens': 263}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-55db04fd-fb66-4d06-a3d4-c1d5164c1b69-0')

In [60]:
chain_with_message_history.invoke(
    {"input": "What was my original question?"},
    {"configurable": {"session_id": "foo"}},
)

AIMessage(content='Your original question was about the conditions for eligibility for the EDG grant in Singapore.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 277, 'total_tokens': 294}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09f9e058-9eea-47e7-81f3-a8063e6afafc-0')

In [ ]:
# Training Langchain Model with More data (10 webpages)

In [ ]:
# Finetuning Langchain model to return relevant forms ON TOP of returning the relevant info